In [1]:
##################################################################
############### Alex Mizway / CS-499 / Databases #################
##################################################################
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
import mysql
import base64

# Importing AnimalShelter class 
from AnimalShelter_AMizway import AnimalShelterMySQL

###########################
# Data Manipulation / Model
###########################
username = 'root'
password = 'root'
aac = AnimalShelterMySQL()

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(aac.read({}))
#########################
# Dashboard Layout / View
#########################
app = dash.Dash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Center(html.B(html.H1('Alex Mizway|CS-499 Dashboard'))),
    html.Hr(),
    html.Div(

        className='row',
        style={'display': 'flex'},
        children=[
            html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue'),
            html.Button(id='submit-button-two', n_clicks=0, children='Mountain or Wilderness Rescue'),
            html.Button(id='submit-button-three', n_clicks=0, children='Disaster Rescue or Individual Tracking'),
            html.Button(id='submit-button-four', n_clicks=0, children='reset')
        ]

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        page_size=100,
        style_table={'height': '300px', 'overflowY': 'auto', 'overflowX': 'auto'},
        style_header={
            'backgroundColor': 'rgb(230,230,230)',
            'fontWeight': 'bold'
        },
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto'
        },

        #tooltips that we are going to use on the table so that we know what information we are looking at
        tooltip={i: {
            'value': i,
            'use_with': 'both'  # both refers to header & data cell
        } for i in df.columns},
        tooltip_delay=0,
        tooltip_duration=None,

        #sorting features that we are going to use
        sort_action='native',
        sort_mode='multi',
        filter_action='native',
        editable=False,
        column_selectable=False,
        row_selectable='single',
        row_deletable=False,
        selected_rows=[],

    ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',

                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])


#############################################
# Interaction Between Components / Controller
#############################################


@app.callback(Output('datatable-id', 'data'), [Input('submit-button-one', 'n_clicks'),
                                               Input('submit-button-two', 'n_clicks'),
                                               Input('submit-button-three', 'n_clicks'),
                                               Input('submit-button-four', 'n_clicks')])
def update_dashboard(bt1, bt2, bt3, bt4):
    query = "SELECT * FROM aac"
    conditions = []

    if bt1 > 0:
        conditions.append(
            "breed IN ('Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland') AND sex_upon_outcome = 'Intact Female' AND age_upon_outcome_in_weeks <= 26 AND age_upon_outcome_in_weeks >= 156")
    elif bt2 > 0:
        conditions.append(
            "breed IN ('German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler') AND sex_upon_outcome = 'Intact Male' AND age_upon_outcome_in_weeks <= 26 AND age_upon_outcome_in_weeks >= 156")
    elif bt3 > 0:
        conditions.append(
            "breed IN ('Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler') AND sex_upon_outcome = 'Intact Male' AND age_upon_outcome_in_weeks <= 20 AND age_upon_outcome_in_weeks >= 300")
    elif bt4 > 0:
        pass  # No additional conditions for reset

    if conditions:
        query += " WHERE " + " AND ".join(conditions)

    df = pd.DataFrame.from_records(aac.read_from_query(query))
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')

    return data


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [
        {
            'if': {'column_id': i},
            'background_color': '#D2F3FF'
        } for i in selected_columns
    ]


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    if not viewData:
        return []  # If there's no data, return an empty graph or an appropriate message

    # Replace 'values', 'names' with appropriate columns from the dataset
    names = viewData.get('animal_type')  # Replace 'names_column_name' with the actual column name

    # Check if values and names are available and not empty
    if names:
        fig = px.pie(names=names, title='Percentage of breeds available')
        return dcc.Graph(figure=fig)
    else:
        return []  # Return an empty graph if the required columns are missing or empty


@app.callback(Output('map-id', "children"), [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    if not viewData:
        return []  # If there's no data, return an empty map or an appropriate message

    # Assuming your viewData contains latitude and longitude columns,
    # replace 'latitude_column' and 'longitude_column' with actual column names
    column_lat = 'location_lat'
    column_long = 'location_long'

    # Check if latitude and longitude data are available
    if column_lat in viewData and column_long in viewData:
        map_children = [
            dl.Map(
                style={'width': '1000px', 'height': '500px'},
                center=[viewData[column_lat][0], viewData[column_long][0]],
                zoom=15,
                children=[
                    dl.TileLayer(id="base-layer-id"),
                    dl.Marker(
                        position=[viewData[column_lat][0], viewData[column_long][0]],
                        children=[
                            dl.Tooltip(viewData['breed']),
                            dl.Popup([
                                html.H1("Animal Name"),
                                html.P(viewData['name'])
                            ])
                        ]
                    )
                ]
            )
        ]
        return map_children
    else:
        return []  # Return an empty map if the required columns are missing or empty


if __name__ == '__main__':
    app.run(mode='external')